# LangChain PromptTemplate과 LCEL 체인 구성하기

이 노트북에서는 **PromptTemplate**을 사용하여 프롬프트를 구조화하고, **LCEL(LangChain Expression Language)**로 체인을 구성하는 방법을 알아봅니다.

## PromptTemplate이란?

프롬프트 템플릿은 동적으로 변경되는 값을 포함한 프롬프트를 정의하는 방법입니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                    PromptTemplate 개념                             │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반 문자열:                                                     │
│   "거대 언어 모델에 대해 알려주세요"                               │
│   → 고정된 질문, 재사용 불가                                       │
│                                                                    │
│   PromptTemplate:                                                  │
│   "{topic}에 대해 알려주세요"                                     │
│   → {topic} 부분만 바꿔서 다양한 질문 생성 가능                    │
│                                                                    │
│   활용:                                                            │
│   • Context + Question 패턴 (RAG의 기본)                           │
│   • 역할 지정 + 작업 지시                                          │
│   • 입력 형식 표준화                                               │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정 (Colab)

In [ ]:
import subprocess
import time

# zstd 설치 (Ollama 설치의 사전 요구 사항)
!apt-get install -y zstd

# Ollama 설치
!curl -fsSL https://ollama.com/install.sh | sh

# 백그라운드에서 Ollama 서버 실행
subprocess.Popen(['ollama', 'serve'])

time.sleep(3)

In [ ]:
# 모델 다운로드 및 패키지 설치
!ollama pull llama3.2
!pip install -q langchain langchain-ollama

# 2. PromptTemplate 단독 사용

먼저 PromptTemplate만 사용해서 프롬프트가 어떻게 생성되는지 확인합니다.

**원본 Python 파일 (04.prompt.py)의 동작:**
- `PromptTemplate.from_template()` - 문자열에서 `{변수명}` 패턴을 자동 감지
- `template.invoke({...})` - 변수에 값을 채워서 완성된 프롬프트 반환

In [ ]:
from langchain_core.prompts import PromptTemplate

# 원본 코드와 동일한 템플릿 생성
template = PromptTemplate.from_template('''아래 작성한 컨텍스트(Context)를 기반으로
    질문(Question)에 대답하세요. 제공된 정보로 대답할 수 없는 질문이라면 "모르겠어요." 라고 답하세요.

Context: {context}

Question: {question}

Answer: ''')

# 템플릿에 값을 채워서 프롬프트 생성 (LLM 호출 없이)
response = template.invoke({
    'context': '''거대 언어 모델(LLM)은 자연어 처리(NLP) 분야의 최신 발전을 이끌고 있습니다.
    거대 언어 모델은 더 작은 모델보다 우수한 성능을 보이며, NLP 기능을 갖춘 애플리케이션을 개발하는 개발자들에게
    매우 중요한 도구가 되었습니다. 개발자들은 Hugging Face의 `transformers` 라이브러리를
    활용하거나, `openai` 및 `cohere` 라이브러리를 통해 OpenAI와 Cohere의 서비스를 이용하여
    거대 언어 모델을 활용할 수 있습니다.
    ''',
    'question': '거대 언어 모델은 어디서 제공하나요?'
})

print("=== 생성된 프롬프트 ===")
print(response)

**출력 결과 설명:**

위 코드에서 `template.invoke()`는 LLM을 호출하지 않습니다. 단지 `{context}`와 `{question}` 자리에 값을 채워서 완성된 프롬프트 문자열을 반환합니다.

이것이 바로 **원본 Python 파일(04.prompt.py)**의 동작입니다.

---

# 3. PromptTemplate + LLM 체인 (LCEL)

실제로 LLM에게 질문하려면 템플릿과 모델을 연결해야 합니다.

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama

# 1. 프롬프트 템플릿 생성
template = PromptTemplate.from_template('''아래 작성한 컨텍스트(Context)를 기반으로
    질문(Question)에 대답하세요. 제공된 정보로 대답할 수 없는 질문이라면 "모르겠어요." 라고 답하세요.

Context: {context}

Question: {question}

Answer: ''')

# 2. Ollama 모델 초기화
model = ChatOllama(model='llama3.2', temperature=0)

# 3. LCEL 체인 구성: template → model
chain = template | model

# 4. 체인 실행
response = chain.invoke({
    'context': '''거대 언어 모델(LLM)은 자연어 처리(NLP) 분야의 최신 발전을 이끌고 있습니다.
    거대 언어 모델은 더 작은 모델보다 우수한 성능을 보이며, NLP 기능을 갖춘 애플리케이션을 개발하는 개발자들에게
    매우 중요한 도구가 되었습니다. 개발자들은 Hugging Face의 `transformers` 라이브러리를
    활용하거나, `openai` 및 `cohere` 라이브러리를 통해 OpenAI와 Cohere의 서비스를 이용하여
    거대 언어 모델을 활용할 수 있습니다.
    ''',
    'question': '거대 언어 모델은 어디서 제공하나요?'
})

print("=== LLM 응답 ===")
print(response.content)

---

## 정리: PromptTemplate

### 원본 코드 (04.prompt.py) vs 확장 코드

| 구분 | 원본 코드 | 확장 코드 |
|------|----------|----------|
| 목적 | 프롬프트 생성만 | 프롬프트 생성 + LLM 호출 |
| 코드 | `template.invoke({...})` | `(template \| model).invoke({...})` |
| 반환값 | StringPromptValue (문자열) | AIMessage (LLM 응답) |

### 핵심 개념

```python
# 1. 템플릿 생성 - {변수명} 자동 감지
template = PromptTemplate.from_template("Context: {context}\nQuestion: {question}")

# 2. 템플릿만 사용 (프롬프트 문자열 생성)
prompt_string = template.invoke({'context': '...', 'question': '...'})

# 3. LCEL 체인 (템플릿 + LLM)
chain = template | model
response = chain.invoke({'context': '...', 'question': '...'})
```

### 코드 변경점 (OpenAI → Ollama)

```python
# 원본 (OpenAI 사용 시)
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model='gpt-4o-mini')

# 변경 (Ollama 사용)
from langchain_ollama import ChatOllama
model = ChatOllama(model='llama3.2')
```

### 다음 단계

- **05.prompt-model.py**: 프롬프트와 모델을 LCEL로 연결하는 패턴
- **06.chat-prompt.py**: ChatPromptTemplate으로 역할별 메시지 구성